In [1]:
%load_ext sql
%config SqlMagic.feedback = False
%matplotlib inline

import os
import pandas as pd
import numpy as np
from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz

# Get env vars from local.env
host = %env DB_HOSTNAME
user = %env DB_USERNAME
password = %env DB_PASSWORD
db = %env DB_NAME
token =  %env MAPBOX_API_KEY

# Connection URL to our local MySQL DB
#%sql mysql+mysqldb://{user}:{password}@{host}/{db}?charset=utf8

In [2]:
#subs = %sql SELECT * FROM submissions WHERE from_mlab = 0
#subs = subs.DataFrame()
#subs['latitude'].dropna(inplace=True)
#subs['longitude'].dropna(inplace=True)
#subs.describe()

In [3]:
columns = [
    'id', 'testing_for', 'address', 'zip_code', 'provider',
    'connected_with', 'monthly_price', 'provider_down_speed',
    'provider_price', 'actual_down_speed', 'actual_price', 'rating',
    'completed', 'created_at', 'updated_at', 'latitude', 'longitude',
    'ping', 'actual_upload_speed', 'test_id', 'ip_address', 'hostname',
    'from_mlab', 'area_code', 'test_type', 'census_code',
    'upload_median', 'download_median', 'census_status', 'test_date',
    'country_code', 'region', 'county', 'accuracy', 'location',
    'census_block'
]

# Load and filter CSV in chunks (less memory, require 5GB to load full dataset) to just from_mlab = 0 (SUA submissions)
iter_csv = pd.read_csv('./data/9035f7b8-2d2f-4de0-a816-4067e1ae8fd8.csv', header=None, names=columns,  iterator=True, chunksize=1000)
sua = pd.concat([chunk[chunk['from_mlab'] == 0] for chunk in iter_csv])

# Cleanup
sua['rating'] = sua['rating'].replace(0.0,  np.nan)

In [18]:
#lane = sua[sua['county'] == 41039].copy()
lane = sua.copy()

lane.fillna(value=np.nan, inplace=True)
#lane = lane[lane.accuracy.notnull() & lane.latitude.notnull() & lane.longitude.notnull()]
lane = lane[lane.latitude.notnull() & lane.longitude.notnull()]

In [19]:
lane.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,1035.000000,556.000000,487.000000,461.000000,1038.000000,461.000000,657.000000,1038.000000,1038.000000,1038.000000,...,1038.000000,1038.0,0.0,1.014000e+03,0.0,0.0,1037.000000,2.710000e+02,0.0,9.340000e+02
mean,95060.711111,75.444245,139.765914,5.615965,57.573911,27.309241,4.225266,0.982659,43.747552,-121.367168,...,27.516599,0.0,NaN,4.054141e+10,NaN,NaN,40057.395371,1.426643e+04,NaN,4.107419e+14
std,10964.223158,38.922822,489.760627,12.642658,83.934222,111.440545,1.949304,0.130602,2.133904,7.974292,...,82.677206,0.0,NaN,4.470706e+09,NaN,NaN,5606.976028,1.176771e+05,NaN,1.555655e+13
min,6473.000000,0.000000,0.000000,0.010000,0.030000,0.040000,1.000000,0.000000,20.907700,-157.992500,...,0.000000,0.0,NaN,6.037462e+09,NaN,NaN,4019.000000,6.000000e+00,NaN,1.601195e+14
25%,97401.000000,50.000000,12.000000,0.540000,7.730000,1.150000,3.000000,1.000000,44.020125,-123.160700,...,1.927500,0.0,NaN,4.103900e+10,NaN,NaN,41039.000000,5.000000e+01,NaN,4.103900e+14
50%,97405.000000,65.000000,60.000000,1.350000,29.690000,3.020000,4.000000,1.000000,44.050000,-123.092700,...,5.820000,0.0,NaN,4.103900e+10,NaN,NaN,41039.000000,5.000000e+01,NaN,4.103900e+14
75%,97448.000000,90.000000,150.000000,4.500000,72.457500,10.890000,6.000000,1.000000,44.088475,-123.021500,...,11.597500,0.0,NaN,4.103900e+10,NaN,NaN,41039.000000,1.062000e+03,NaN,4.103900e+14
max,99352.000000,200.000000,10000.000000,120.000000,828.200000,1428.570000,7.000000,1.000000,47.548000,-72.839700,...,918.310000,0.0,NaN,5.502500e+10,NaN,NaN,55025.000000,1.488703e+06,NaN,5.306301e+14


In [21]:
# Create a geojson file export from a Pandas dataframe
#df_to_geojson(lane, filename='points.geojson', properties=['accuracy'], lat='latitude', lon='longitude', precision=3)
df_to_geojson(lane, filename='points.geojson', properties=[], lat='latitude', lon='longitude', precision=3)

{'type': 'file', 'filename': 'points.geojson', 'feature_count': 1038}

In [22]:
# Generate data breaks and color stops from colorBrewer
color_breaks = [0, 10, 25, 50, 100, 1000]
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

# Create the viz from the dataframe
viz = CircleViz('points.geojson',
    access_token=token,
    height='800px',
#    color_property = "accuracy",
    color_stops = color_stops,
    center = (-123.09, 44.082),
    zoom = 8,
    below_layer = 'waterway-label'
)
viz.show()